In [2]:
import pandas as pd
import sqlite3

# Задача 1
У некоторого сервиса существует подписка, которая позволяет юзерам получать кэшбек за покупки. Подписка конечна, но может быть продлена
Надо написать запрос, чтобы посчитать количество пользователей, у которых доля покупок с активной подпиской составляет больше 80% от общего числа их заказов

Структура таблиц:

market_orders

    order_id – ID заказа
    utc_creation_dttm – дата создания заказа в формате YYYY-MM-DD hh:mm:ss
    user_id – ID пользователя

active_plus

    user_id – ID пользователя
    utc_dttm_from – дата в формате YYYY-MM-DD hh:mm:ss, начиная с которой подписка была активна
    utc_dttm_to – дата в формате YYYY-MM-DD hh:mm:ss, до которой подписка была активна

Гарантируется, что промежутки активности подписок не пересекаются


Сделаем тестовую таблицу

In [8]:
con = sqlite3.connect("mydatabase.db")
cursor = con.cursor()

In [9]:
def select(sql):
    return pd.read_sql(sql, con)

In [10]:
df = pd.DataFrame({'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                   'utc_creation_dttm': ['2020-01-01', '2020-10-01', '2020-03-01', '2020-04-01', '2020-05-01',
                                         '2020-01-01', '2020-07-01', '2020-09-01', '2020-11-01', '2020-06-01',
                                         '2020-12-01', '2020-5-01'],
                   'user_id': [1, 1, 2, 3, 4, 5, 5, 5, 6, 7, 8, 9]})
df.to_sql('market_orders', con, index=False, if_exists='replace')

In [11]:
df2 = pd.DataFrame({'user_id': [1, 3, 5, 5],
                    'utc_dttm_from': ['2020-01-01', '2020-01-01', '2020-01-01', '2020-08-01'],
                    'utc_dttm_to': ['2020-02-01', '2020-12-01', '2020-02-01', '2020-12-01']})
df2.to_sql('active_plus', con, index=False, if_exists='replace')

In [13]:
sql = '''

select
    count(user_id) as plus_buyers_counter
from
    (
    select
        market_orders.user_id,
        count(order_id) as total_orders,
        max(plus_orders) as plus_orders
    from
        market_orders
    left join (
                select
                    active_plus.user_id,
                    count(order_id) as  plus_orders
                from
                    active_plus
                left join
                    market_orders on market_orders.user_id = active_plus.user_id
                where
                    utc_creation_dttm between utc_dttm_from and utc_dttm_to
                group by
                    active_plus.user_id) as counter on counter.user_id = market_orders.user_id

    group by
    market_orders.user_id)

where
  cast(plus_orders as real) / cast(total_orders as real) > 0.8


'''
select(sql)

,plus_buyers_counter
0,1


# Задача 2
orders

    order_id,
    promocode_id

promocodes

    promocode_id,
    name,
    discount


Необходимо вывести

1) Долю заказов с промокодами
2) Самый популярный промокод (название) и число его использований

In [14]:
con = sqlite3.connect("mydatabase.db")
cursor = con.cursor()

In [15]:
def select(sql):
  return pd.read_sql(sql, con)

In [16]:
orders = pd.DataFrame({
                       'order_id':[1, 2, 3, 4, 5, 6, 7],
                       'promocode_id':[None, None, 2, 2, 3, 3, 1]
})

orders.to_sql('orders',con,index=False,if_exists='replace')

In [17]:
promocodes = pd.DataFrame({
                           'promocode_id':[1, 2, 3],
                           'name':['first', 'second', 'third'],
                           'discount':[0, 0, 0]
})

promocodes.to_sql('promocodes',con,index=False,if_exists='replace')

In [18]:
query = '''
select
    top_promo, promo_use_count, promo_orders_share
from
    (select
        name as top_promo, count as promo_use_count
    from
        promocodes
    left join (
        select
            count(order_id) as count, orders.promocode_id
        from
            orders
        group by
            orders.promocode_id
        ) as merged on merged.promocode_id = promocodes.promocode_id

    where
        promo_use_count =(
            select
                max(count)
            from(
                select
                    count(order_id) as count
                from
                    orders
                group by
                    orders.promocode_id

        ))) left join
            (select
                round(cast(count(promocode_id) as real) / count(order_id), 3) as promo_orders_share
            from
                orders)

'''
select(query)

,top_promo,promo_use_count,promo_orders_share
0,second,2,0.714
1,third,2,0.714


# Задача 3

consumption

    coffee_point_id INT,
    cookies INT

buildings

    coffee_point_id INT,
    coffee_point_name VARCHAR(32),
    office_id INT,
    office_name VARCHAR(32)


Необходимо вывести топ-10 офисов по потреблению печенек среди офисов, которые потребляют менее 1000 печенек

In [19]:
consumption = pd.DataFrame({
                       'coffee_point_id':[1, 2, 3, 4, 5, 6, 7],
                       'cookies':[600, 100, 200, 200, 300, 300, 1000]
})

consumption.to_sql('consumption',con,index=False,if_exists='replace')

In [20]:
buildings = pd.DataFrame({
                    'coffee_point_id':[1, 2, 3, 4, 5, 6, 7],
                    'coffee_point_name':['a', 'b', 'c', 'd', 'e', 'f', 'g'],
                    'office_id':[1, 1, 2, 2, 3, 3, 1],
                    'office_name':['red', 'red', 'blue', 'blue', 'green', 'green', 'red']
})

buildings.to_sql('buildings',con,index=False,if_exists='replace')

In [21]:
query = '''
select
    *
from (
    select
        office_name, sum(cookies) as office_consumption
    from
        buildings
    left join
        consumption on buildings.coffee_point_id = consumption.coffee_point_id
    group by
        office_name
    order by
        office_consumption desc
    )
where
    office_consumption < 1000
limit
    10
'''
select(query)

,office_name,office_consumption
0,green,600
1,blue,400
